# Preprocessing Dataset #2

## Import the data

In [ ]:
import pandas as pd
# 1. Load the datasets into Pandas dataframes
names2=['A1-Time','A2','A3','A4','A5','A6','A7','A8','A9','LABEL']
#Full training set will be split into training and validation
dataset2_trn_full = pd.read_csv('shuttle_trn.csv', index_col=False, names=names2, sep=' ', engine='python') 
dataset2_test_full = pd.read_csv('shuttle_tst.csv', index_col=False, names=names2, sep=' ', engine='python')


## Data visulization & Cleaning
### Note: Data is complete, no missing entries

In [ ]:
# Visualizing data to have a better idea of the data
# Plotting frequency of different attribute values since there are only 9 (+ 1 for assigned labels)
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy import stats
import numpy as np

# Visualize all numerical categories
dataset2_trn_full.hist(figsize=(20, 14))

'''
#removing outliers
subset=dataset2_trn.loc[:,'A2':'A9']
print(subset)
z_scores = stats.zscore(subset)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
print(filtered_entries)
dataset2_trn = dataset2_trn[filtered_entries]'''

print(dataset2_trn_full.describe())



In [ ]:
# Normalizing data since scales of some attributes are drastically different
from sklearn import preprocessing
names2_x=['A1-Time','A2','A3','A4','A5','A6','A7','A8','A9']
x_d2=dataset2_trn_full.loc[:,'A1-Time':'A9']
y_d2=dataset2_trn_full.loc[:,'LABEL']

'''min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_train)
normalized_dataset_2 = pd.DataFrame(x_scaled, columns=names2)

normalized_dataset_2.hist(figsize=(20, 14))'''

scaler = preprocessing.StandardScaler()

# fit the scaler 
scaler.fit(x_d2)

# transform the data
x_train_normal = scaler.transform(x_d2)

normalized_xd2 = pd.DataFrame(x_train_normal, columns=names2_x)

normalized_xd2.hist(figsize=(20, 14))
#y_d2.hist(figsize=(20,14))
print(normalized_xd2.describe())




## One-hot coding
### No need for one-hot coding since all attributes and labels are numerical values

# Experiments Dataset #2

## Cross Validation

### Split training data into training and validation, start with 9:1 ratio (90% traning, 10% validation)

In [ ]:

def splitTraining(p_val,num_instances):
    n_val=num_instances//p_val
    np.random.seed(5)
    inds = np.random.permutation(num_instances)
    x_val2,y_val2=normalized_xd2.loc[inds[:n_val],:],y_d2.loc[inds[:n_val]]
    x_train2,y_train2=normalized_xd2.loc[inds[n_val:],:],y_d2.loc[inds[n_val:]]
    
    return n_val, x_val2,y_val2, x_train2, y_train2

### Cross validation method

In [ ]:
def cross_validate(total_nval, n_folds=5):
    #get the number of data samples in each split
    n_val = total_nval // n_folds
    np.random.seed(0)
    inds = np.random.permutation(total_nval)
    inds = []
    for f in range(n_folds):
        tr_inds = []
        #get the validation indexess
        val_inds = list(range(f * n_val, (f+1)*n_val))
        #get the train indexes
        if f > 0:
            tr_inds = list(range(f*n_val))
        if f < n_folds - 1:
            tr_inds = tr_inds + list(range((f+1)*n_val, total_nval))
        #The yield statement suspends function’s execution and sends a value back to the caller
        #but retains enough state information to enable function to resume where it is left off
        yield tr_inds, val_inds

### Setting up training/validation data sets and variables needed for hyperparameter analysis

In [ ]:
(trn_instances, num_features) = dataset2_trn_full.shape
num_folds = 5

#split the data
(n_val_25, x_val2_25,y_val2_25, x_train2_75, y_train2_75)=splitTraining(25,int(0.3*trn_instances))
(n_val_50, x_val2_50,y_val2_50, x_train2_50, y_train2_50)=splitTraining(50,int(0.3*trn_instances))
(n_val_75, x_val2_75,y_val2_75, x_train2_25, y_train2_25)=splitTraining(75,int(0.3*trn_instances))

#converting training and validation dataframe sets to numpy arrays
x_val2_25=x_val2_25.to_numpy()
y_val2_25=y_val2_25.to_numpy()
x_train2_75=x_train2_75.to_numpy()
y_train2_75=y_train2_75.to_numpy()


x_val2_50=x_val2_50.to_numpy()
y_val2_50=y_val2_50.to_numpy()
x_train2_50=x_train2_50.to_numpy()
y_train2_50=y_train2_50.to_numpy()

x_val2_75=x_val2_75.to_numpy()
y_val2_75=y_val2_75.to_numpy()
x_train2_25=x_train2_25.to_numpy()
y_train2_25=y_train2_25.to_numpy()

#define a function for the accuracy
accuracy= lambda y, yh: np.mean(y==yh)

# KNN

## Evaluating best number of K neighbors

In [ ]:
from sklearn import neighbors


def find_bestK(n_val,x_train,y_train,x_val,y_val,title):
    K_list=range(1,100)
    acc_train,acc_valid = np.zeros(len(K_list)), np.zeros((len(K_list), num_folds))
    #Evaluating best number of K neighbors
    for i, K in enumerate(K_list):
        #Find the validation errors for num_folds splits for a given K
        model = neighbors.KNeighborsRegressor(n_neighbors=K)
        model = model.fit(x_train, y_train)
        acc_train[i]= accuracy(y_train, model.predict(x_train))
        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
            model = neighbors.KNeighborsRegressor(n_neighbors=K)
            model = model.fit(x_val[tr,:], y_val[tr])
            acc_valid[i, f] = accuracy(y_val[val], model.predict(x_val[val])) 
    
    plt.plot(K_list, acc_train,  label='train')
    plt.errorbar(K_list, np.mean(acc_valid, axis=1), np.std(acc_valid, axis=1), label='validation')
    plt.legend()
    plt.xlabel('K (number of neighbours)')
    plt.ylabel('Accuracy')
    plt.show()
    plt.title(title)


## Evaluating neighbor weights

In [ ]:
from sklearn import neighbors


#Find best K with weighted neighbors
def find_bestK_weighted(n_val,x_train,y_train,x_val,y_val,title):
    K_list=range(1,100)
    acc_train,acc_valid = np.zeros(len(K_list)), np.zeros((len(K_list), num_folds))
    #Evaluating best number of K neighbors
    for i, K in enumerate(K_list):
        #Find the validation errors for num_folds splits for a given K
        model = neighbors.KNeighborsRegressor(n_neighbors=K,weights='distance')
        model = model.fit(x_train, y_train)
        acc_train[i]= accuracy(y_train, model.predict(x_train))
        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
            model = neighbors.KNeighborsRegressor(n_neighbors=K,weights='distance')
            model = model.fit(x_val[tr,:], y_val[tr])
            acc_valid[i, f] = accuracy(y_val[val], model.predict(x_val[val])) 
    
    plt.plot(K_list, acc_train,  label='train')
    plt.errorbar(K_list, np.mean(acc_valid, axis=1), np.std(acc_valid, axis=1), label='validation')
    plt.legend()
    plt.xlabel('K (number of neighbours)')
    plt.ylabel('Accuracy')
    plt.show()
    plt.title(title)

## Evaluating best distance to use (Euclidean or Manhattan)

## Verifying custom cross-validation results against cross-validation package

In [ ]:
from sklearn.model_selection import cross_val_score
#add sklearn cross val method for verification purposes


# Decision Tree

## Evaluating best split criterion without max depth limitation

In [ ]:
from sklearn import tree

#Find best split criteria

def find_best_crit(n_val,x_train,y_train,x_val,y_val,title):
        criteria_list=["gini", "entropy"] 
        acc_train,acc_valid = np.zeros(len(criteria_list)), np.zeros((len(criteria_list), num_folds))

        model = tree.DecisionTreeClassifier()
        model = model.fit(x_train, y_train)
        acc_train[0]= accuracy(y_train, model.predict(x_train))


        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
                model = tree.DecisionTreeClassifier()
                model = model.fit(x_val[tr,:], y_val[tr])
                acc_valid[0,f] = accuracy(y_val[val], model.predict(x_val[val])) 


        model = tree.DecisionTreeClassifier(criterion="entropy")
        model = model.fit(x_train, y_train)
        acc_train[1]= accuracy(y_train, model.predict(x_train))

        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
                model = tree.DecisionTreeClassifier(criterion="entropy")
                model = model.fit(x_val[tr,:], y_val[tr])
                acc_valid[1,f] = accuracy(y_val[val], model.predict(x_val[val])) 



        x=[1,2]

        plt.subplot(1,2,1)
        plt.scatter(x,acc_train, label='training')
        plt.xlabel('Gini Index=1                Entropy=2')
        plt.ylabel('Accuracy')
        plt.title('Training')

        plt.subplot(1,2,2)
        plt.scatter(x,np.mean(acc_valid, axis=1),label='validation')
        #plt.errorbar(criteria_list, np.mean(err_valid, axis=1), np.std(err_valid, axis=1), label='Validation')
        plt.title('Validation')
        plt.xlabel('Gini Index=1                Entropy=2')
        plt.ylabel('Accuracy')

        plt.tight_layout(pad=5)
        plt.show()
        plt.title(title)

## Evaluating best max depth limitation with Gini index

In [ ]:
from sklearn import tree


#Find best max depth using Gini index

def find_maxD_gini(n_val,x_train,y_train,x_val,y_val,title):
    max_depth=range(4,8)

    acc_train,acc_valid = np.zeros(len(max_depth)), np.zeros((len(max_depth), num_folds))


    for i, d in enumerate(max_depth):
        #Find the validation errors for num_folds splits for a given max_depth
        model = tree.DecisionTreeClassifier(criterion='gini',max_depth=d)
        model = model.fit(x_train, y_train)
        acc_train[i]= accuracy(y_train, model.predict(x_train))
        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
            model = tree.DecisionTreeClassifier(criterion='gini',max_depth=d)
            model = model.fit(x_val[tr,:], y_val[tr])
            acc_valid[i, f] = accuracy(y_val[val], model.predict(x_val[val])) 
    
    plt.plot(max_depth, acc_train,  label='Train')
    plt.errorbar(max_depth, np.mean(acc_valid, axis=1), np.std(acc_valid, axis=1), label='Validation')
    plt.legend()
    plt.xlabel('Max depth')
    plt.ylabel('Accuracy')
    plt.show()


## Evaluating best max depth limitation with Entropy 

In [ ]:
from sklearn import tree

#Find best max depth using Gini index

def find_minsamp_entropy(n_val,x_train,y_train,x_val,y_val,title):
    max_depth=range(4,8)

    acc_train,acc_valid = np.zeros(len(max_depth)), np.zeros((len(max_depth), num_folds))

    #Evaluating value for max depth
    for i, d in enumerate(max_depth):
        #Find the validation errors for num_folds splits for a max_depth
        model = tree.DecisionTreeClassifier(criterion='entropy',max_depth=d)
        model = model.fit(x_train, y_train)
        acc_train[i]= accuracy(y_train, model.predict(x_train))
        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
            model = tree.DecisionTreeClassifier(criterion='entropy',max_depth=d)
            model = model.fit(x_val[tr,:], y_val[tr])
            acc_valid[i, f] = accuracy(y_val[val], model.predict(x_val[val])) 
    
    plt.plot(max_depth, acc_train,  label='Train')
    plt.errorbar(max_depth, np.mean(acc_valid, axis=1), np.std(acc_valid, axis=1), label='Validation')
    plt.legend()
    plt.xlabel('Max depth')
    plt.ylabel('Accuracy')
    plt.show()

Conclusion: Entropy gives best results, max depth does not affect MSE (if we were too choose max depth, then 4 (?) should be a good number)
Note: 0 MSE on training error (?)

## Evaluating the best minimum number of samples using Entropy

In [ ]:
#Find best min number of samples using Entropy

def find_minsamp_entrop(n_val,x_train,y_train,x_val,y_val,title):
    min_sample=range(3,8)

    acc_train,acc_valid = np.zeros(len(min_sample)), np.zeros((len(min_sample), num_folds))

    #Evaluating value for max depth
    for i, d in enumerate(min_sample):
        #Find the validation errors for num_folds splits for a max_depth
        model = tree.DecisionTreeClassifier(criterion='entropy',max_depth=d)
        model = model.fit(x_train, y_train)
        acc_train[i]= accuracy(y_train, model.predict(x_train))
        for f, (tr, val) in enumerate(cross_validate(n_val, num_folds)):
            model = tree.DecisionTreeClassifier(criterion='entropy',max_depth=d)
            model = model.fit(x_val[tr,:], y_val[tr])
            acc_valid[i, f] = accuracy(y_val[val], model.predict(x_val[val])) 
    
    plt.plot(min_sample, acc_train,  label='Train')
    plt.errorbar(min_sample, np.mean(acc_valid, axis=1), np.std(acc_valid, axis=1), label='Validation')
    plt.legend()
    plt.xlabel('Max depth')
    plt.ylabel('Accuracy')
    plt.show()

## Calling training models with hyperparameter evaluations on growing subsets of training/validation data

# Experiments on test data

## Setting up test data

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import neighbors

x_d2_test=dataset2_test_full.loc[:,'A1-Time':'A9']
y_d2_test=dataset2_test_full.loc[:,'LABEL']

scaler = preprocessing.StandardScaler()

# fit the scaler 
scaler.fit(x_d2_test)

# transform the data
x_test_normal = scaler.transform(x_d2_test)

normalized_xd2_test = pd.DataFrame(x_test_normal, columns=names2_x)

x_test2=normalized_xd2_test.to_numpy()
y_test2=y_d2_test.to_numpy()

'''normalized_xd2.hist(figsize=(20, 14))
#y_d2.hist(figsize=(20,14))
print(normalized_xd2.describe())'''

## KNN (K = 2, weighted neighbors (or not? performs similarly with and without), Euclidian distance)

In [ ]:
best_model_KNN = neighbors.KNeighborsClassifier(n_neighbors=2,weights='distance')
best_model_KNN.fit(x_test2, y_test2)
test_pred = best_model_KNN.predict(x_test2)

acc = np.mean(test_pred == y_test2)
print(confusion_matrix(y_test2, test_pred), f'{acc:.3f}')

## Decision Tree (Splite criteria = Entropy, max depth = 5)

In [ ]:
best_model_dt = tree.DecisionTreeClassifier(criterion='entropy',max_depth=5)
best_model_dt.fit(x_test2, y_test2)
test_pred_dt = best_model_dt.predict(x_test2)

acc = np.mean(test_pred_dt == y_test2)
print(confusion_matrix(y_test2, test_pred_dt), f'{acc:.3f}')